<a href="https://colab.research.google.com/github/visiont3lab/machine-learning-course/blob/main/regression/Regression_Code_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline Regressione

## Importare librerie e funzioni

In [ ]:
# Create a class to select numerical or categorical columns 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error  # MSE
from sklearn.metrics import mean_absolute_error # MAE
from sklearn.metrics import median_absolute_error # MedAE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RANSACRegressor, SGDRegressor, HuberRegressor, TheilSenRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
import joblib
import seaborn as sn
import matplotlib.pyplot as plt
import random
import numpy as np
import plotly.graph_objects as go
import pickle
import json
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn import datasets
from google.colab import data_table
import seaborn as sns
import plotly.express as px

## Caricare il dataset

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/visiont3lab/flask-static-website/master/dataset_ready.csv")
display(df.head())
Y = df.pop("Specific Gravity").values
X = df.values

,Age,Albumin,Anemia-no,Anemia-yes,Appetite-good,Appetite-poor,Bacteria-notpresent,Bacteria-present,Blood Glucose Random,Blood Pressure,Blood Urea,Class,Coronary Artery Disease-no,Coronary Artery Disease-yes,Diabetes Mellitus-no,Diabetes Mellitus-yes,Hemoglobin,Hypertension-no,Hypertension-yes,Packed Cell Volume,Pedal Edema-no,Pedal Edema-yes,Potassium,Pus Cell clumps-notpresent,Pus Cell clumps-present,Pus Cell-abnormal,Pus Cell-normal,Red Blood Cell Count,Red Blood Cells-abnormal,Red Blood Cells-normal,Serum Creatinine,Sodium,Specific Gravity,Sugar,White Blood Cell Count
0,48.0,4.0,0,1,0,1,1,0,117.0,70.0,56.0,1,1,0,1,0,11.2,0,1,32.0,0,1,2.5,0,1,1,0,3.9,0,1,3.8,111.0,1.005,0.0,6700.0
1,53.0,2.0,0,1,0,1,1,0,70.0,90.0,107.0,1,1,0,0,1,9.5,0,1,29.0,1,0,3.7,0,1,1,0,3.7,1,0,7.2,114.0,1.020,0.0,12100.0
2,63.0,3.0,1,0,0,1,1,0,380.0,70.0,60.0,1,1,0,0,1,10.8,0,1,32.0,0,1,4.2,0,1,1,0,3.8,1,0,2.7,131.0,1.010,0.0,4500.0
3,68.0,3.0,1,0,0,1,0,1,157.0,80.0,90.0,1,0,1,0,1,5.6,0,1,16.0,0,1,6.4,0,1,1,0,2.6,0,1,4.1,130.0,1.010,2.0,11000.0
4,61.0,2.0,0,1,0,1,1,0,173.0,80.0,148.0,1,0,1,0,1,7.7,0,1,24.0,0,1,5.2,1,0,1,0,3.2,1,0,3.9,135.0,1.015,0.0,9200.0


 ## Training

In [ ]:
# Shuffle
#X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.20,shuffle=True, random_state=25)

# Model selection
models = {}
models["Linear"]        = LinearRegression()
models["Huber"]         = HuberRegressor(max_iter=1000)
#models["TheilSen"]      = TheilSenRegressor()
models["Ridge"]         = Ridge()
models["Lasso"]         = Lasso()
models["ElasticNet"]    = ElasticNet()
models["KNN"]           = KNeighborsRegressor(n_neighbors=5)
models["DecisionTree"]  = DecisionTreeRegressor()
models["SVR"]           = SVR(gamma="auto")
models["AdaBoost"]      = AdaBoostRegressor(n_estimators=50)
models["GradientBoost"] = GradientBoostingRegressor(n_estimators=100)
models["RandomForest"]  = RandomForestRegressor(n_estimators=100)
models["ExtraTrees"]    = ExtraTreesRegressor(n_estimators=100)

# Cors
k_fold  = KFold(n_splits=5, random_state=5,shuffle=True)

model_results = []
model_names   = []
for name in list(models.keys()):
  
  # Model selection
  model_names.append(name)
  # Definisco il modello
  pipe = Pipeline([
      ("sc", StandardScaler()), # fit # mean, std
      ("pca", PCA(0.98)), # fit # mean, std
      #("poly", PolynomialFeatures(2)), # fit # mean, std
      ("model", models[name]) # trovare i parametri a0 a1 a2 
  ])
  c = 0 

  # Cross validation
  results = []
  for train_index, test_index in k_fold.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train = X[train_index]
    y_train = Y[train_index]
    X_test = X[test_index]
    y_test = Y[test_index]

    pipe.fit(X_train,y_train)
    y_hat_test = pipe.predict(X_test)
    err = np.sqrt( mean_squared_error(y_hat_test, y_test)) # rmse
    #err = mean_absolute_error(y_hat_test, y_test)         # mae
    results.append(err)
    print(name + ": insieme: "+ str(c) + " RMSE: ", err)
    c = c+1

  model_results.append(results)

In [ ]:
  fig = go.Figure()
  for name,res in zip(model_names,model_results):    
      fig.add_trace(go.Box(y=res,name=name, boxpoints='all'))
  fig.show()

### Rialleno su tutti i dati

In [ ]:
## Rialleno --> Su tutto il dataset
pipe = Pipeline([
    ("sc", StandardScaler()), # fit # mean, std
    ("pca", PCA(0.98)), # fit # mean, std
    #("poly", PolynomialFeatures(2)), # fit # mean, std
    ("model", models["KNN"]) # trovare i parametri a0 a1 a2 
])
# Su tutti i dati
pipe.fit(X,Y)
joblib.dump(pipe, "model.pkl")

['model.pkl']

## Visualizzazione

In [ ]:
m =joblib.load("model.pkl")
Y_hat = m.predict(X)

# Visualizazione
x = np.linspace(start=0,stop=len(y_test),num=len(y_test))
fig = go.Figure()
fig.add_trace( go.Scatter( x=x, y=Y.flatten(),mode="markers+lines", name="Y") )
fig.add_trace( go.Scatter( x=x, y=Y_hat.flatten(),mode="markers+lines", name="Y_hat") )
fig.update_layout(title_text="Results", hovermode="x")
fig.show()